In [1]:
from diffie_hellman import (alice_generate_keys, bob_generate_keys, alice_compute_shared_secret, bob_compute_shared_secret,
                          generate_vigenere_key, generate_transposition_key)
from hmac_ import demonstrate_collision, demonstrate_length_extension, hmac_xor, hmac_sha256
import hashlib
from ciphers import encrypt_cbc, decrypt_cbc
from ratchet import simulate_conversation_single_ratchet, simulate_conversation_double_ratchet

In [2]:
# 1536-bit MODP Group Prime (from RFC 3526)
p = int("FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1"
             "29024E088A67CC74020BBEA63B139B22514A08798E3404DD"
             "EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245"
             "E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED"
             "EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3D"
             "C2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F"
             "83655D23DCA3AD961C62F356208552BB9ED529077096966D"
             "670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF", 16)

g = 2  # Generator for this group
BLOCK_SIZE = 64


In [3]:
print("Starting Diffie-Hellman Key Exchange Simulation...\n")

a, A = alice_generate_keys(p, g)
print(f"Alice's public key (A): {A}")

b, B = bob_generate_keys(p, g)
print(f"Bob's public key (B): {B}")


shared_secret_alice = alice_compute_shared_secret(a, B, p)
print(f"Alice's computed shared secret: {shared_secret_alice}")
shared_secret_bob = bob_compute_shared_secret(b, A, p)
print(f"Bob's computed shared secret: {shared_secret_bob}")

assert shared_secret_alice == shared_secret_bob, "Shared secrets do not match!"
print("\nShared secret successfully established between Alice and Bob.")

Starting Diffie-Hellman Key Exchange Simulation...

Alice's public key (A): 567724676873400396334710425854505260238160551900302505990137477070034574023816997636030393799039520168987759282138851040361910193029909432529712610812977908410835086691238570749353520307287233593988611031541915716756919521460287151753897609680572451736345236319824898277759436611975095149257226008979789768939592554855783401818799460970798961695319078412683282719975626733875480029051801735808635531964552636713698260683003501186362709741079403701589253050469724
Bob's public key (B): 1432832094808031895968631866833445636079383066249849905316404047022925320818104051635327951891166221757249722432827072364413370224012184866605222222688782315144621974117803233985702367796167787502200025937116980061771631260703907300078063598347154120313485633436765169588987581179685800982166841766048780242178299979829189588450246732068651092128047219670084927870695243518955123602326668825167525068532669629779418309433883585102878737372

In [4]:
# Generate Vigenère key from shared secret
vigenere_key = generate_vigenere_key(shared_secret_alice)
print(f"Vigenère Key: {vigenere_key}")

# Generate Transposition key from shared secret
transposition_key = generate_transposition_key(shared_secret_alice)
print(f"Transposition Key: {transposition_key}")

Vigenère Key: EAFCBFEC
Transposition Key: 53241


# XOR-HMAC

In [5]:
print("Deriving HMAC key from the shared secret...")
hmac_key = shared_secret_alice.to_bytes((shared_secret_alice.bit_length() + 7) // 8, byteorder='big')
print(f"Derived HMAC key: {hmac_key.hex()}\n")

message_str = "Hi Bob, this is a secure message from Alice."
message = message_str.encode('utf-8')
print(f"Alice is sending the message: {message}")
    
hmac_tag = hmac_xor(hmac_key, message)
hmac_hex = hmac_tag.hex()
print(f"Message HMAC: {hmac_hex}\n")

# Concatenate the message and HMAC as a string with '||' delimiter
delimiter = "||"
concatenated_message = message_str + delimiter + hmac_hex
print(f"Concatenated message sent to Bob: {concatenated_message}\n")

print("Bob receives the concatenated message and proceeds to verify the HMAC...")

try: # Split using the last delimiter in the message (if more than one delimiter is present)
    received_message_str, received_hmac_hex = concatenated_message.rsplit(delimiter, 1)
except ValueError:
    print("Error: Invalid message format. Delimiter missing.")
    exit(1)

received_message = received_message_str.encode('utf-8')
received_hmac = bytes.fromhex(received_hmac_hex)

print(f"Received message: {received_message_str}")
print(f"Received HMAC: {received_hmac_hex}\n")

computed_hmac = hmac_xor(hmac_key, received_message)
computed_hmac_hex = computed_hmac.hex()
print(f"Computed HMAC on the received message: {computed_hmac_hex}\n")

if received_hmac_hex == computed_hmac_hex:
    print("HMAC verification successful: The message is authentic and has not been tampered with.")
else:
    print("HMAC verification failed: The message might have been tampered with or is not authentic.")

Deriving HMAC key from the shared secret...
Derived HMAC key: eac5d72ffe1e9b7937ee7df39b245a102c043b1e1d7bead67e31bd21dcd3911fdd8edeaadea4b3228d02ae77315f6e1a5dd0b39e7e48a4448d43f3dd274b2d6e4ad63a4fbae9b5f3deab5b284d63d4135b6f08dfa07c762cd2350ce1c5cd46e51a2b6a1e6c4f2e4604a6605dabba45581b82992eb60121677a9f733a0547d437d0699965c7eba0f03ce78ab1606c01b4e4d292011b3b1df686c98a298af0a42a597176b3831136b0092003fd559b804cb63dda00fa6236745b4f9e670e4f81ba

Alice is sending the message: b'Hi Bob, this is a secure message from Alice.'
Message HMAC: 39

Concatenated message sent to Bob: Hi Bob, this is a secure message from Alice.||39

Bob receives the concatenated message and proceeds to verify the HMAC...
Received message: Hi Bob, this is a secure message from Alice.
Received HMAC: 39

Computed HMAC on the received message: 39

HMAC verification successful: The message is authentic and has not been tampered with.


In [6]:
demonstrate_collision()
demonstrate_length_extension()

Demonstration 1: Collision vulnerability
HMAC for 'b'Hello, World!'': 2d
HMAC for 'b'Olleh, Dlrow!'': 2d
HMACs are the same: True

Demonstration 2: Length extension attack
Original HMAC: 6c
Extended HMAC: 7f
Forged HMAC:   7f
Forged HMAC matches extended HMAC: True


# Solution: use HMAC-SHA256

In [7]:
print("Deriving HMAC key from the shared secret...")
shared_secret_bytes = shared_secret_alice.to_bytes((shared_secret_alice.bit_length() + 7) // 8, byteorder='big')
hmac_key = hashlib.sha256(shared_secret_bytes).digest()

print(f"Derived HMAC key: {hmac_key.hex()}\n")

message_str = "Hi Bob, this is a secure message from Alice."
message = message_str.encode('utf-8')
print(f"Alice is sending the message: {message_str}")

hmac_tag = hmac_sha256(hmac_key, message)
hmac_hex = hmac_tag.hex()
print(f"Message HMAC: {hmac_hex}\n")

# Concatenate the message and HMAC as a string with '||' delimiter
delimiter = "||"
concatenated_message = message_str + delimiter + hmac_hex
print(f"Concatenated message sent to Bob: {concatenated_message}\n")

print("Bob receives the concatenated message and proceeds to verify the HMAC...")

try: # Split using the last delimiter in the message (if more than one delimiter is present)
    received_message_str, received_hmac_hex = concatenated_message.rsplit(delimiter, 1)
except ValueError:
    print("Error: Invalid message format. Delimiter missing.")
    exit(1)

received_message = received_message_str.encode('utf-8')
received_hmac = bytes.fromhex(received_hmac_hex)

print(f"Received message: {received_message_str}")
print(f"Received HMAC: {received_hmac_hex}\n")

computed_hmac = hmac_sha256(hmac_key, received_message)
computed_hmac_hex = computed_hmac.hex()
print(f"Computed HMAC on the received message: {computed_hmac_hex}\n")

if received_hmac_hex == computed_hmac_hex:
    print("HMAC verification successful: The message is authentic and has not been tampered with.")
else:
    print("HMAC verification failed: The message might have been tampered with or is not authentic.")


Deriving HMAC key from the shared secret...
Derived HMAC key: 4eaf3c99b4f47ecf59729822730460c2bf279e8d63f6be6486a77d856ce8a2d0

Alice is sending the message: Hi Bob, this is a secure message from Alice.
Message HMAC: d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Concatenated message sent to Bob: Hi Bob, this is a secure message from Alice.||d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Bob receives the concatenated message and proceeds to verify the HMAC...
Received message: Hi Bob, this is a secure message from Alice.
Received HMAC: d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Computed HMAC on the received message: d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

HMAC verification successful: The message is authentic and has not been tampered with.


# Ciphering

In [8]:
shared_secret_bytes = shared_secret_alice.to_bytes((shared_secret_alice.bit_length() + 7) // 8, byteorder='big')
hmac_key = hashlib.sha256(shared_secret_bytes).digest()
vigenere_key = generate_vigenere_key(shared_secret_alice)
transposition_key = generate_transposition_key(shared_secret_alice)
    
message_str = "Hi Bob, this is a secure message from Alice."
print(f"Alice is sending the message:\n{message_str}\n")

hmac_tag = hmac_sha256(hmac_key, message)
hmac_hex = hmac_tag.hex()

delimiter = "||"
concatenated_message = message_str + delimiter + hmac_hex
# Encrypt the concatenated message
ciphertext = encrypt_cbc(concatenated_message, transposition_key, vigenere_key)
print(f"Concatenated message:\n{concatenated_message}\n")
print(f"Encrypted message sent to Bob:\n{ciphertext}\n")

print("Bob receives the encrypted message and proceeds to decrypt it...")
# Decrypt the message
decrypted_message = decrypt_cbc(ciphertext, transposition_key, vigenere_key)
print(f"Decrypted message:\n{decrypted_message}\n")

try: # Split using the last delimiter in the message (if more than one delimiter is present)
    received_message_str, received_hmac_hex = decrypted_message.rsplit(delimiter, 1)
except ValueError:
    print("Error: Invalid message format. Delimiter missing.")
    exit(1)

received_message = received_message_str.encode('utf-8')
received_hmac = bytes.fromhex(received_hmac_hex)

print(f"Received message: {received_message}")
print(f"Received HMAC: {received_hmac.hex()}\n")

computed_hmac = hmac_sha256(hmac_key, received_message)
print(f"Computed HMAC on the received message: {computed_hmac.hex()}\n")

if received_hmac == computed_hmac:
    print("HMAC verification successful: The message is authentic and has not been tampered with.")
else:
    print("HMAC verification failed: The message might have been tampered with or is not authentic.")

Alice is sending the message:
Hi Bob, this is a secure message from Alice.

Concatenated message:
Hi Bob, this is a secure message from Alice.||d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Encrypted message sent to Bob:
´©fc«r··Ê  Õ)kOi­¸¶©~·èÞ ÞÕÍ(31Zru®«g[Ô÷e#2¹j£ty·Ôñg
Ø,3KmÀur®­aEÒõ·ÒÔ	ó3A>wrp®b@Ôö

Bob receives the encrypted message and proceeds to decrypt it...
Decrypted message:
Hi Bob, this is a secure message from Alice.||d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Received message: b'Hi Bob, this is a secure message from Alice.'
Received HMAC: d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

Computed HMAC on the received message: d8e18b73584bfe646a7ed1a6f58d98ddbd7a2cbf6332ab322e473f3e30bd3a0d

HMAC verification successful: The message is authentic and has not been tampered with.


# Ratchet

In [9]:
simulate_conversation_single_ratchet()

Performing Diffie-Hellman Key Exchange...

Deriving HMAC key from shared secret...
Derived HMAC key: 9cafb61150c14a36a5b66038c891683f956ce1c148346b458c0e7a3881389d0d

Chain Key initialization for Alice and Bob completed.
Alice's Chain Key: 9cafb61150c14a36a5b66038c891683f956ce1c148346b458c0e7a3881389d0d
Bob's Chain Key:   9cafb61150c14a36a5b66038c891683f956ce1c148346b458c0e7a3881389d0d


Alice sends: Hi Bob, how are you?
Concatenated Message:
 yx~wxsu©nfÖ    
¤supÏ«¤väßÞfÞõ¥|t§­~bÒ fÙ	ø¥©u¦Às¯¡nÓ 	·
v©wfw`p¢kâé

Bob receives:
 yx~wxsu©nfÖ    
¤supÏ«¤väßÞfÞõ¥|t§­~bÒ fÙ	ø¥©u¦Às¯¡nÓ 	·
v©wfw`p¢kâé
Valid HMAC: True
Decrypted message: Hi Bob, how are you?

Alice's Chain Key after sending: b091c361fa2edcf38613dea09d483965d642663c46c5319671474bd45247665e0000000000000000000000000000000000000000000000000000000000000000
Bob's Chain Key after receiving: b091c361fa2edcf38613dea09d483965d642663c46c5319671474bd45247665e00000000000000000000000000000000000000000

In [10]:
simulate_conversation_double_ratchet()

Performing Diffie-Hellman Key Exchange...
Shared secret:
928328669905669211202717078757567278279421195391331068053470809347459537468752949538302883485698620401306336297761936010960442591541408637774895494774875662807598236542293208949491934099669168836741653445251787877591771293763242723197598133952967404823456591092014982753885055555987872744358216706484415563403387831037348104276901132919808343667932286450814706878935585190150594731662952090292567637656341004789609180466709132729247502572412714186205889305652166
Deriving HMAC key from shared secret...
Derived HMAC key: b93b6fbcb1c4cd0b859e7fb13f8be00bb8b141b0280c688e3efc49f27ef1acee

Chain Key initialization for Alice and Bob completed.
Alice's Chain Key: b93b6fbcb1c4cd0b859e7fb13f8be00bb8b141b0280c688e3efc49f27ef1acee
Bob's Chain Key:   b93b6fbcb1c4cd0b859e7fb13f8be00bb8b141b0280c688e3efc49f27ef1acee


Alice sends: Hi Bob, how are you?
Concatenated Message:
 |w|wzuwukÚ 
 þ ©wqt{vw{kÕÝ÷ò  Å¦Á{|z`5ÚÓcÕºÓ R­ Ë¯¯uÇ	·Ö½